**Import necessary libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

**Load and preprocess the training data**

In [2]:
train_data = pd.read_excel('01 Train Data.xlsx')

**Remove duplicate rows based on 'Email ID'**

In [3]:
df = pd.DataFrame(train_data)

df = df.drop_duplicates(subset='Email ID', keep='first')

df.head()

,First Name,Email ID,Quantity,Price Tier,Ticket Type,Attendee #,Group,Order Type,Currency,Total Paid,...,Attendee Status,College Name,How did you come to know about this event?,"Specify in ""Others"" (how did you come to know about this event)",Designation,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
0,ANIKET,aniket@xyz.com,1,NaN,Art of Resume Building,2.213855e+09,NaN,Free Order,USD,0,...,Attending,D Y PATIL INSTITUTE OF MCA AND MANAGEMENT AKUR...,Email,NaN,Students,NaN,6.7,2,5,Placed
1,Dhanshree,dhanshree@xyz.com,1,NaN,Art of Resume Building,2.213859e+09,NaN,Free Order,USD,0,...,Attending,AP SHAH INSTITUTE OF TECHNOLOGY,Others,College,Students,NaN,8.2,3,2,Not placed
2,Dhiraj,dhiraj@xyz.com,1,NaN,Art of Resume Building,2.213862e+09,NaN,Free Order,USD,0,...,Attending,Don Bosco College of Engineering Fatorda Goa,Email,NaN,Students,NaN,6.5,4,3,Not placed
3,Pooja,pooja@xyz.com,1,NaN,Art of Resume Building,2.213988e+09,NaN,Free Order,USD,0,...,Attending,Pillai College of Engineering New Panvel,Email,NaN,Students,NaN,8.7,2,5,Not placed
4,Aayush,aayush@xyz.com,1,NaN,Art of Resume Building,2.214567e+09,NaN,Free Order,USD,0,...,Attending,St Xavier's College,Instagram | LinkedIn | Cloud Counselage Website,NaN,Students,NaN,9.1,3,5,Placed


In [4]:
df.isna().sum()

First Name                                                            0
Email ID                                                              0
Quantity                                                              0
Price Tier                                                         1987
Ticket Type                                                           0
Attendee #                                                          222
Group                                                              1987
Order Type                                                            0
Currency                                                            222
Total Paid                                                            0
Fees Paid                                                           222
Eventbrite Fees                                                       0
Eventbrite Payment Processing                                         0
Attendee Status                                                 

In [5]:
df.shape

(1987, 23)

**Drop specified columns from the DataFrame**

In [6]:
columns_to_remove = [
    'Price Tier', 'Group', 'Email ID', 'Quantity', 'Total Paid', 'Fees Paid',
    'Eventbrite Fees', 'Eventbrite Payment Processing', 'Order Type', 'Currency',
    'Attendee #', 'How did you come to know about this event?',
    'Specify in "Others" (how did you come to know about this event)','First Name', 'College Name','Ticket Type','Designation','Attendee Status'
]
df.drop(columns_to_remove, axis=1, inplace=True)
df.head()


,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
0,NaN,6.7,2,5,Placed
1,NaN,8.2,3,2,Not placed
2,NaN,6.5,4,3,Not placed
3,NaN,8.7,2,5,Not placed
4,NaN,9.1,3,5,Placed


In [7]:
df.shape

(1987, 5)

In [8]:
df.isnull().sum()

Year of Graduation    1707
CGPA                     0
Speaking Skills          0
ML Knowledge             0
Placement Status      1321
dtype: int64

**Map 'Placement Status' values: 'Placed' to 1, 'Not placed' to 0, and fill NaN with 2, then convert to integers**

In [9]:
df['Placement Status'] = df['Placement Status'].map({'Placed': 1, 'Not placed': 0}).fillna(2).astype(int)

In [10]:
df.isnull().sum()

Year of Graduation    1707
CGPA                     0
Speaking Skills          0
ML Knowledge             0
Placement Status         0
dtype: int64

In [11]:
df['Placement Status'].unique()

array([1, 0, 2])

In [12]:
df.head()

,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
0,NaN,6.7,2,5,1
1,NaN,8.2,3,2,0
2,NaN,6.5,4,3,0
3,NaN,8.7,2,5,0
4,NaN,9.1,3,5,1


In [14]:
df['Year of Graduation'].unique()

array([nan, '2021', 'Friend', '2023', 'Third year', 'Second year', '2022',
       '2020', '1999', '2016', 'Final Year', 'Fourth Year', 'B.Tech',
       '2019', 'Pursuing', 'AISSMS', 'Bachelor of Engineering ', '2024'],
      dtype=object)

**Define values to remove from 'Year of Graduation' column and
Filter the DataFrame to exclude rows with specified values**

In [15]:
values_to_remove = ['AISSMS', 'Bachelor of Engineering ','Friend','Pursuing', 'B.Tech']

df = df[~df['Year of Graduation'].isin(values_to_remove)]

**Fill NaN values in 'Year of Graduation' with -1**

In [16]:
df['Year of Graduation'] = df['Year of Graduation'].fillna(-1)

C:\Users\zoeu\AppData\Local\Temp\ipykernel_17480\2904799340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year of Graduation'] = df['Year of Graduation'].fillna(-1)


**Define a custom mapping for 'Year of Graduation'**

In [18]:
custom_mapping = {'Third year': '3','Second year': '2', 'Final Year': '4', 'Fourth Year': '4'}

df['Year of Graduation'].replace(custom_mapping, inplace=True)

df['Year of Graduation'] = pd.to_numeric(df['Year of Graduation'], errors='coerce')

C:\Users\zoeu\AppData\Local\Temp\ipykernel_17480\962370073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year of Graduation'].replace(custom_mapping, inplace=True)
C:\Users\zoeu\AppData\Local\Temp\ipykernel_17480\962370073.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year of Graduation'] = pd.to_numeric(df['Year of Graduation'], errors='coerce')


In [19]:
df['Year of Graduation'].unique()

array([  -1, 2021, 2023,    3,    2, 2022, 2020, 1999, 2016,    4, 2019,
       2024], dtype=int64)

In [20]:
df.tail()

,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
4829,-1,9.1,5,4,2
4831,-1,7.4,3,2,2
4832,-1,7.3,4,4,2
4834,-1,9.2,4,2,2
4836,-1,7.9,2,5,2


**Load and preprocess the test data**

In [24]:
test_data = pd.read_excel('02 Test Data.xlsx')

dt = pd.DataFrame(test_data)

In [25]:
dt.shape

(3796, 23)

**Perform the same preprocessing on the test dataset as done above for the above train data**

In [26]:
dt = dt.drop_duplicates(subset='Email ID', keep='first')

dt.head()

,First Name,Email ID,Quantity,Price Tier,Ticket Type,Attendee #,Group,Order Type,Currency,Total Paid,...,Attendee Status,College Name,How did you come to know about this event?,"Specify in ""Others"" (how did you come to know about this event)",Designation,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
0,Sahil,sahil@xyz.com,1,NaN,Hello ML and DL,2.293940e+09,NaN,Free Order,USD,0,...,Attending,"symbiosis institute of technology, pune",Whatsapp,NaN,Students,NaN,7.8,3,3,NaN
1,Amrita,amrita@xyz.com,1,NaN,Hello ML and DL,2.293941e+09,NaN,Free Order,USD,0,...,Attending,"mit academy of engineering ,alandi",Whatsapp,NaN,Students,NaN,9.1,3,3,NaN
2,Mamta,mamta@xyz.com,1,NaN,Hello ML and DL,2.293941e+09,NaN,Free Order,USD,0,...,Attending,a. c. patil college of engineering,Whatsapp,NaN,Students,NaN,6.9,2,2,NaN
3,Bhagyashri,bhagyashri@xyz.com,1,NaN,Hello ML and DL,2.293946e+09,NaN,Free Order,USD,0,...,Attending,wilson college,Others,NaN,Students,NaN,8.4,4,4,NaN
4,Divyanshu,divyanshu@xyz.com,1,NaN,Hello ML and DL,2.293956e+09,NaN,Free Order,USD,0,...,Attending,"ld college of engineering, ahmedabad, gujarat",Whatsapp,NaN,Students,NaN,6.7,5,5,NaN


In [ ]:
columns_to_remove = [
    'Price Tier', 'Group', 'Email ID', 'Quantity', 'Total Paid', 'Fees Paid',
    'Eventbrite Fees', 'Eventbrite Payment Processing', 'Order Type', 'Currency',
    'Attendee #', 'How did you come to know about this event?',
    'Specify in "Others" (how did you come to know about this event)', 'First Name', 'College Name', 'Ticket Type',
    'Designation', 'Attendee Status'
]

In [28]:
dt.drop(columns_to_remove, axis=1, inplace=True)

In [30]:
values_to_remove_t = ['AISSMS', 'Bachelor of Engineering ','Friend','Pursuing', 'B.Tech','i am student in 12 commerce']

dt = dt[~dt['Year of Graduation'].isin(values_to_remove_t)]

In [29]:
dt.shape

(2321, 5)

In [31]:
dt['Year of Graduation'] = dt['Year of Graduation'].fillna(-1)

In [32]:
custom_mapping_t = {'Third year': '3','Second year': '2', 'Final Year': '4', 'Fourth Year': '4','Last Year':'4'}

dt['Year of Graduation'].replace(custom_mapping_t, inplace=True)

dt['Year of Graduation'] = pd.to_numeric(dt['Year of Graduation'], errors='coerce')

In [33]:
dt['Year of Graduation'].unique()

array([  -1,    2, 2023, 2021, 2019, 2022,    3,    4, 2020, 2016, 1999,
       2013, 2015, 2000, 2017, 2024, 2018], dtype=int64)

**Split the dataset into training and testing sets**

In [21]:
X = df.drop(columns=['Placement Status']) 
y = df['Placement Status'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Initialize a Random Forest Classifier with a random seed for reproducibility**

In [22]:
clf = RandomForestClassifier(random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Predict the target variable for the test data and calculate the accuracy of the model**

In [23]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.6237373737373737


**Prepare the test data for making predictions**

In [34]:
X_test = dt.drop(columns=['Placement Status'])

test_predictions = clf.predict(X_test) 

In [35]:
unique_values = list(set(test_predictions))

print(unique_values)

[0, 1, 2]


**Update the 'Placement Status' column in the test data with the predictions**

In [36]:
dt['Placement Status'] = test_predictions

In [37]:
dt.head()

,Year of Graduation,CGPA,Speaking Skills,ML Knowledge,Placement Status
0,-1,7.8,3,3,2
1,-1,9.1,3,3,2
2,-1,6.9,2,2,2
3,-1,8.4,4,4,2
4,-1,6.7,5,5,2


In [38]:
placement_status_counts = dt['Placement Status'].value_counts()

print(placement_status_counts)

Placement Status
2    2112
0     154
1      46
Name: count, dtype: int64


**Save the test data with predictions to an Excel file**

In [41]:
dt.to_excel('test_data_with_predictions.xlsx', index=False)